In [1]:
# Import libraries and data 
import scipy.io as sio
from sklearn.model_selection import TimeSeriesSplit
import copy
import pickle
import numpy as np
from scipy.signal import find_peaks
from os.path import dirname, join as pjoin
import datetime
import csv
import math
import sys
sys.path.insert(1,'../')
import Tools.data_processing as dp
import Tools.data_movement as dm 
from auxiliary_functions import get_candidates, detect_flat_plat, detect_stage_rises


fDOM_raw_data = dm.read_in_preprocessed_timeseries('../Data/converted_data/julian_format/fDOM_raw_10.1.2011-9.4.2020.csv')
stage_data = dm.read_in_preprocessed_timeseries('../Data/converted_data/julian_format/stage_10.1.11-1.1.19.csv')
turb_data = dm.read_in_preprocessed_timeseries('../Data/converted_data/julian_format/turbidity_raw_10.1.2011_9.4.2020.csv')
stage_data = dp.align_stage_to_fDOM(fDOM_raw_data, stage_data)

### Detect Stage Rises and Process 
##### Process stage rises so that each index contains value representing distance to next stage rise in both positive and negative directioin

In [4]:
# Get stage rises
s_indices = detect_stage_rises(stage_data[:,1])

# Process stage rises so that each index displays distance to next stage rise in positive and negative direction
y = s_indices.shape[0] -1 
s_indexed = np.zeros((s_indices.shape[0],2))
x_count = -1 
y_count = -1
for x in range(s_indices.shape[0]):
    # X Block 
    
    # When x encounters first stage rise, start x counter
    if x_count == -1 and s_indices[x] == 1:
        x_count = 0
    if x_count != -1:
        if s_indices[x] == 1:
            x_count = 0
            s_indexed[x,0] = x_count
        else:
            x_count += 1
            s_indexed[x,0] = x_count
    else:
        s_indexed[x,0] = -1
            
    # Y Block
    if y_count == -1 and s_indices[y] == 1:
        y_count = 0
    if y_count != -1:
        if s_indices[y] == 1:
            y_count = 0
            s_indexed[y,1] = y_count
        else:
            y_count += 1
            s_indexed[y,1] = y_count
    else: 
        s_indexed[y,1] = -1
        
    y-=1

### Get Candidate Turb PP Peaks and Process 
##### For each peak, determine how far away from a stage rise is is in both the negative and postive temporal directione
##### Import ground truth values

In [5]:
    # Get turb and fDOM peaks
fDOM_cand_params = {'prom' : [8,None],
                    'width': [5, None],
                    'wlen' : 300,
                    'dist' : 20,
                    'rel_h': .6}

turb_cand_params = {'prom' : [6,None],
                    'width': [None, None],
                    'wlen' : 200,
                    'dist' : 1,
                    'rel_h': .6}

fDOM_peaks, fDOM_props = get_candidates(fDOM_raw_data, fDOM_cand_params)
turb_peaks, turb_props = get_candidates(turb_data, turb_cand_params)

# Remove peaks that occur during a flat plateau 

turb_peaks, turb_props = dp.delete_missing_data_peaks(turb_data, turb_peaks, turb_props, '/Users/zachfogg/Desktop/DB-SRRW/Data/misc/flat_plat_ranges.txt')
# Convert peaks and props to useable structure and assign values from s_indexed
# Each entry = [index_of_peak, left_ips, right_ips, X, Y, flag]
turb_cand = [[peak, math.floor(turb_props['left_ips'][i]), math.ceil(turb_props['right_ips'][i]),s_indexed[peak,0], s_indexed[peak,1] ,turb_props['prominences'][i]] for i,peak in enumerate(turb_peaks)]

# Import ground truth values 
truth_fname = '/Users/zachfogg/Desktop/DB-SRRW/Data/manual_annotating_data/processed_data/turb/julian_time/turb_pp_0k-300k-2_labeled'

with open(truth_fname, 'r', newline = '') as f:
    
        reader = csv.reader(f, delimiter = ',')
        # truth entries in form: ['timestamp_of_peak', 'value_of_peak','label_of_peak','idx_of_peak']
        next(reader)
        truths = [[float(row[0]), float(row[1]), row[2], int(row[3])] for row in reader] 
        f.close()   

assert(len(truths) == len(turb_cand))

### Hyperparameter Search While Preforming K-Fold Nested Cross Validation on Rule Based Classifier
- parameter "x" refers to the number of data points before the next stage rise a point is
- parameter "y" refers to the number of data points after the previous stage rise a point is 


In [6]:
# Define Hyperparameter ranges and training parameters and helper function
iterations = 70000
num_splits = 5

turb_peak_prom_threshold = 50 # Threshold for turb peak at which we must see fDOM interference 

x_bounds = [0, 30]
y_bounds = [0, 30]

intf_range_1_bounds = [-10,2]
intf_range_2_bounds = [-5,3]
intf_range_3_bounds = [-2,10]

intf_threshold_bounds = [-2,2]


accumulated_test_metrics = {}

accumulated_test_results = {}

accumulated_best_params = {}

train_test_split_indices = TimeSeriesSplit(num_splits).split(turb_cand)

def check_for_fDOM_interference(fDOM, peak_indx, intf_params):
    """
    Given the index of a turb peak, check fDOM for 
    interference using given parameters 
    """
    # Calculate mean tangent line across range_1
    mean_tan_1 = np.mean(np.diff(fDOM[(peak_indx + intf_params['inft_range_1'][0]):(peak_indx + intf_params['inft_range_1'][1]),1]))
    # Calculate mean tangent line across range_2
    mean_tan_2 = np.mean(np.diff(fDOM[(peak_indx + intf_params['inft_range_2'][0]):(peak_indx + intf_params['inft_range_2'][1]),1]))
    # Calculate mean tangent line across range_3
    mean_tan_3 = np.mean(np.diff(fDOM[(peak_indx + intf_params['inft_range_3'][0]):(peak_indx + intf_params['inft_range_3'][1]),1])) 
    # Compare means 
    if (mean_tan_1 * intf_params['intf_t1']) > (mean_tan_2) or (mean_tan_2) < (mean_tan_3 * intf_params['intf_t2']):
        return True 
    return False

def classify_turb_peaks(peaks, params):
    results = []
    for peak in peaks:
        if(peak[3] != -1 and peak[3] <= params['x']) or (peak[4] !=-1 and peak[4] <= params['y']):
            if peak[5] > turb_peak_prom_threshold: 
                intf_result = check_for_fDOM_interference(fDOM_raw_data, peak[0], params)
                if intf_result: 
                    results.append([peak[0], 'NPP'])
                else: 
                    results.append([peak[0], 'PP'])
            else: 
                results.append([peak[0], 'NPP'])
        else: 
            results.append([peak[0], 'PP'])
    return results

def label_positives_negatives(predictions, truths):
    TP = TN = FP = FN = 0
    results = []
    
    for i in range(len(predictions)):
        prediction = predictions[i][1]
        truth = truths[i][2]
        
        if prediction == 'NPP':
            if truth == 'NPP':
                TN +=1
                results.append(predictions[i].append('TN'))
            else: 
                FN+=1
                results.append(predictions[i].append('FN'))
        else:
            if truth == 'NPP':
                FP+=1
                results.append(predictions[i].append('FP'))
            else: #TODO: Comeback and evaluate if this makes sense: Algo could predict PP because X/Y was not optimal, while truth was PPP because of interference
                TP+=1
                results.append(predictions[i].append('TP'))
        
    return (TP,TN,FP,FN,results)
    

In [80]:
overall_start = datetime.datetime.now()

split = 1

for train_val_indices, test_indices in train_test_split_indices:
    X_train, y_train = [turb_cand[i] for i in train_val_indices], [truths[i] for i in train_val_indices]
    X_test, y_test = [turb_cand[i] for i in test_indices], [truths[i] for i in test_indices]
    
    max_fold_metric = 0 
    max_result = None 
    
    print("Split: ",split)
    
    split_start = datetime.datetime.now()
    
    for iteration in range(iterations):
        
        # Random grid search for hyperparams 
        params = {}
        
        params['x'] = np.random.randint(x_bounds[0], x_bounds[1]+1)
        params['y'] = np.random.randint(y_bounds[0], y_bounds[1]+1)
#         params['x'] = 1
#         params['y'] = 11
        
        temp = np.random.randint(intf_range_1_bounds[0], intf_range_1_bounds[1])
        params['inft_range_1'] = (temp, np.random.randint(temp, intf_range_1_bounds[1]) + 2)
        
        temp = np.random.randint(intf_range_2_bounds[0], intf_range_2_bounds[1])
        params['inft_range_2'] = (temp, np.random.randint(temp, intf_range_2_bounds[1]) + 2)
        
        temp = np.random.randint(intf_range_3_bounds[0], intf_range_3_bounds[1])
        params['inft_range_3'] = (temp, np.random.randint(temp, intf_range_3_bounds[1]) + 2)
        
        params['intf_t1'] = np.random.uniform(intf_threshold_bounds[0], intf_threshold_bounds[1])
        params['intf_t2'] = np.random.uniform(intf_threshold_bounds[0], intf_threshold_bounds[1])
        
        predictions = classify_turb_peaks(X_train, params)
        
        TP,TN,FP,FN,results = label_positives_negatives(predictions, y_train)
        
        TPR = TP/(TP + FN)
        TNR = TN/(TN + FP)
        
        bal_acc = (TPR + TNR)/2
        
        f1_score = (2 * TP)/((2 * TP) + FP + FN)

        acc = f1_score
        if iteration and iteration % int(iterations/10) == 0: print(" {}/{} ".format(iteration, iterations), end = "")
        if acc > max_fold_metric: 
            max_fold_metric = acc
            max_result = copy.deepcopy(results)
            best_params = copy.deepcopy(params)
            
    # Test best parameters on testing data 
    test_predictions = classify_turb_peaks(X_test, best_params)
    TP,TN,FP,FN,results = label_positives_negatives(test_predictions, y_test)
    
    TPR = TP/(TP + FN)
    TNR = TN/(TN + FP)
    
    bal_acc = (TPR + TNR)/2 
    f1_score = (2 * TP)/((2 * TP) + FP + FN)
    
    print('\nSplit: {}  F1: {:.4f} BA: {:.4f}  Params: {}  TP: {} TN: {} FP: {} FN: {}  Time: {}'.format(split, f1_score, bal_acc, best_params, TP, TN, FP, FN, datetime.datetime.now() - split_start))
    accumulated_test_metrics[split] = [f1_score, bal_acc] # Record test metrics of each split
    accumulated_test_results[split] = copy.deepcopy(results) # Record test results (FP,FN,TP,TN for each datapoint) for each split
    accumulated_best_params[split] = copy.deepcopy(best_params) # Record params uses in testing for each split
    
    split+=1

# Display Resuls 
mean_f1 = 0
mean_ba = 0 

for key in accumulated_test_metrics:
    metrics = accumulated_test_metrics[key]
    mean_f1+=metrics[0]
    mean_ba+=metrics[1]

print("Mean Test F1: ", mean_f1/len(accumulated_test_metrics))
print("Mean Test BA: ", mean_ba/len(accumulated_test_metrics))

print("Training time: ", datetime.datetime.now() - overall_start)

# Pickle params from last fold
with open('./Experiments/turb_PP/best_params.pkl', 'wb') as pck_file:
    pickle.dump(accumulated_best_params[num_splits], pck_file)
    pck_file.close()
    
# Pickle results from last fold 
with open('./Experiments/turb_PP/test_results.pkl', 'wb') as pck_file:
    pickle.dump(accumulated_test_results, pck_file)
    pck_file.close()

# Pickle results from last fold 
with open('./Experiments/turb_PP/test_metrics.pkl', 'wb') as pck_file:
    pickle.dump(accumulated_test_metrics, pck_file)
    pck_file.close()

Split:  1
 7000/70000  14000/70000  21000/70000  28000/70000  35000/70000  42000/70000  49000/70000  56000/70000  63000/70000 
Split: 1  F1: 0.9570 BA: 0.9529  Params: {'x': 9, 'y': 8, 'inft_range_1': (-8, -2), 'inft_range_2': (-5, 1), 'inft_range_3': (8, 11), 'intf_t1': -0.8066638028104025, 'intf_t2': -1.5181968305986993}  TP: 89 TN: 77 FP: 6 FN: 2  Time: 0:00:38.908799
Split:  2
 7000/70000  14000/70000  21000/70000  28000/70000  35000/70000  42000/70000  49000/70000  56000/70000  63000/70000 
Split: 2  F1: 0.9565 BA: 0.9523  Params: {'x': 18, 'y': 1, 'inft_range_1': (-8, 3), 'inft_range_2': (-5, 1), 'inft_range_3': (-2, 10), 'intf_t1': -0.36255907688326783, 'intf_t2': 1.2122392329914708}  TP: 99 TN: 66 FP: 2 FN: 7  Time: 0:01:36.471527
Split:  3
 7000/70000  14000/70000  21000/70000  28000/70000  35000/70000  42000/70000  49000/70000  56000/70000  63000/70000 
Split: 3  F1: 0.9365 BA: 0.9610  Params: {'x': 12, 'y': 2, 'inft_range_1': (-6, 1), 'inft_range_2': (-5, -2), 'inft_range_3'

### Plot results to look for inconsistencies

In [26]:
# Mean Test F1:  0.9447109515545028
# Mean Test BA:  0.9525206551697059
# Training time:  0:11:54.100646
# Best: Params: {'x': 13, 'y': 2, 'inft_range_1': (0, 2), 'inft_range_2': (-4, 1), 'inft_range_3': (9, 11), 
# 'intf_t1': 0.7100971913695409, 'intf_t2': 0.0656586836339943}

test_params = {'x': 13, 'y': 2, 'inft_range_1': (0, 2), 'inft_range_2': (-4, 1), 'inft_range_3': (9, 11), 'intf_t1': 0.7100971913695409, 'intf_t2': 0.0656586836339943}

predictions = classify_turb_peaks(turb_cand, test_params)
        
TP,TN,FP,FN,results = label_positives_negatives(predictions, truths)

f1_score = (2 * TP)/((2 * TP) + FP + FN)
print((TP + TN)/(TP + TN + FN + FP))
print(predictions[0])
print(turb_cand[0])
print("FP: ", FP, "FN: ", FN, "TN: ", TN, " TP: ", TP)

0.9665711556829035
[191, 'NPP', 'TN']
[191, 186, 195, 0.0, 0.0, 8.4696597116]
FP:  21 FN:  14 TN:  515  TP:  497


In [8]:
peak_map = {}
for pred in predictions:
    peak_map[pred[0]] = [pred[1], pred[2]]

In [9]:
# Get stage rises
s_indices = detect_stage_rises(stage_data[:,1])
# fDOM_cand_params = {'prom' : [4,None],
#                     'width': [None, None],   These are the params for fDOM pp cand
#                     'wlen' : 200,
#                     'dist' : 1,
#                     'rel_h': .6}

fDOM_cand_params = {'prom' : [4,None],
                    'width': [None, 2],
                    'wlen' : 200,
                    'dist' : 1,
                    'rel_h': .6}

# turb_cand_params = {'prom' : [6,None],       These are the params that were used to label turb cand 0-100k
#                     'width': [None, None],
#                     'wlen' : 200,
#                     'dist' : 1,
#                     'rel_h': .6}

turb_cand_params = {'prom' : [6,None],
                    'width': [None, 2],
                    'wlen' : 200,
                    'dist' : 1,
                    'rel_h': .5}

# Get fDOM and turb candiate peaks
fDOM_peaks, fDOM_props = get_candidates(fDOM_raw_data, fDOM_cand_params)
turb_peaks, turb_props = get_candidates(turb_data, turb_cand_params)

# Remove peaks that occur during a flat plateau 
turb_flat_plat = detect_flat_plat(turb_data, 100, 40)
turb_flat_plat_indxs = []
for i in range(turb_flat_plat.shape[0]):
    if turb_flat_plat[i] == 1:
        turb_flat_plat_indxs.append(i)

take_indices = []
for i,peak in enumerate(turb_peaks):
    if peak not in turb_flat_plat_indxs:
        take_indices.append(i)

turb_peaks = np.take(turb_peaks, take_indices)
for key in turb_props:
    turb_props[key] = np.take(turb_props[key], take_indices)

# Iterate through peaks and turn into short 3 point "events" by flagging the data point to either side of a peak
# fDOM_events = []
# for peak in fDOM_peaks:
#             fDOM_events.append(np.array((fDOM_raw_data[peak-1], fDOM_raw_data[peak], fDOM_raw_data[peak+1])))
fDOM_events = []
fDOM_lb = []
fDOM_rb = []

for i,peak in enumerate(fDOM_peaks):
            fDOM_events.append(np.array((fDOM_raw_data[peak])))
#             fDOM_lb.append(fDOM_raw_data[fDOM_props['left_bases'][i],0])
#             fDOM_rb.append(fDOM_raw_data[fDOM_props['right_bases'][i],0])
            fDOM_lb.append(fDOM_raw_data[math.floor(fDOM_props['left_ips'][i]),0])
            fDOM_rb.append(fDOM_raw_data[math.ceil(fDOM_props['right_ips'][i]),0])
            
fDOM_lb = list(set(fDOM_lb))
fDOM_lb.sort()
fDOM_rb = list(set(fDOM_rb))
fDOM_rb.sort()

turb_events = []
turb_lb = []
turb_rb = []
for i,peak in enumerate(turb_peaks):
            turb_events.append(np.array((turb_data[peak])))
            turb_lb.append(turb_data[math.floor(turb_props['left_ips'][i]),0])
            turb_rb.append(turb_data[math.ceil(turb_props['right_ips'][i]),0])
            
turb_lb = list(set(turb_lb))
turb_lb.sort()
turb_rb = list(set(turb_rb))
turb_rb.sort()            

fDOM_merged = dp.merge_data(fDOM_raw_data, fDOM_events, 'f_opp', '')
turb_merged = dp.merge_data(turb_data, turb_events, 't_opp', '')

fDOM_merged = dp.merge_additional_data(fDOM_merged, fDOM_lb, 'left_base')
fDOM_merged = dp.merge_additional_data(fDOM_merged, fDOM_rb, 'right_base')

turb_merged = dp.merge_additional_data(turb_merged, turb_lb, 'left_base')
turb_merged = dp.merge_additional_data(turb_merged, turb_rb, 'right_base')


stage_edge_data = dp.stage_rises_to_data(s_indices, stage_data)
stage_data_merged = dp.merge_data(stage_data, stage_edge_data, 'rise','')

In [23]:
for peak in peak_map.keys():
    turb_merged[peak][2] = peak_map[peak][0] + "_" + peak_map[peak][1]

In [24]:
dm.write_data_to_trainset(fDOM_merged,
                          stage_data_merged,
                          turb_merged,
                          '../Data/plot/pred_turb_pp_0k-100k.csv',
                          True,
                          True,
                          0,
                          100000)